<a target="_blank" href="https://colab.research.google.com/github/jingjieyeo/abm_on_colab/blob/main/2_bacteria_chemotaxis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Bacterial Chemotaxis Model
## How bacteria navigate their environment


This Google Colab notebook explores bacterial chemotaxis - the ability of bacteria to sense and move toward or away from chemical gradients in their environment. This is a crucial mechanism that allows bacteria to find nutrients and avoid toxins.


## Setup and Installation


In [ ]:
# Install Mesa and required libraries
!pip install mesa matplotlib ipywidgets

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import random
import math

from mesa import Agent, Model
from mesa.space import ContinuousSpace
from mesa.datacollection import DataCollector
from IPython.display import clear_output, HTML
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

In [ ]:
# Define a chemical gradient environment
class ChemicalGradient:
    """
    Creates a chemical gradient in the environment.
    """
    def __init__(self, width, height, gradient_type="linear", center=None, steepness=1.0):
        """
        Initialize a chemical gradient.

        Args:
            width: Width of the environment
            height: Height of the environment
            gradient_type: Type of gradient ('linear', 'radial', or 'exponential')
            center: Center point of the gradient for radial gradients
            steepness: Controls how steep the gradient is
        """

        self.width = width
        self.height = height
        self.gradient_type = gradient_type
        self.steepness = steepness

        # Set center of gradient (default to center of environment)
        if center is None:
            self.center = (width / 2, height / 2)
        else:
            self.center = center

    def concentration_at(self, position):
        """
        Calculate the chemical concentration at a given position.

        Args:
            position: (x, y) coordinates

        Returns:
            Concentration value between 0 and 1
        """

        x, y = position
        if self.gradient_type == "linear":
            # Linear gradient from left to right
            return x / self.width * self.steepness
        elif self.gradient_type == "radial":
            # Radial gradient decreasing from center
            distance = math.sqrt((x - self.center[0])**2 + (y - self.center[1])**2)
            max_distance = math.sqrt((self.width)**2 + (self.height)**2) / 2
            return max(0, 1 - (distance / max_distance) * self.steepness)
        elif self.gradient_type == "exponential":
            # Exponential gradient from left to right
            return (1 - math.exp(-self.steepness * x / self.width)) / (1 - math.exp(-self.steepness))
        return 0

In [ ]:
# Define the bacterial agent with chemotaxis behavior
class ChemotacticBacterium(Agent):
    """
    A bacterial agent that can sense and move toward chemical gradients.
    """
    def __init__(self, model, speed=0.5, memory_length=3, sensitivity=5):
        """
        Initialize a new bacterial agent with chemotaxis capabilities.

        Args:
            unique_id: Unique identifier for the agent
            model: The model the agent belongs to
            speed: Movement speed (distance per step)
            memory_length: How many past concentration readings to remember
            sensitivity: How sensitive the bacterium is to concentration changes
        """

        super().__init__(model)

        # Movement parameters
        self.speed = speed
        self.heading = random.uniform(0, 2 * math.pi) # Random initial direction
        self.tumble_rate = 0.1 # Base probability of tumbling

        # Chemotaxis parameters
        self.memory_length = memory_length
        self.sensitivity = sensitivity
        self.concentration_history = []

        # State tracking
        self.state = "run" # Can be "run" or "tumble"
        self.tumble_timer = 0

    def sense_environment(self):
        """
        Sense the chemical concentration at the current position.
        """

        concentration = self.model.chemical_gradient.concentration_at(self.pos)

        # Add to history and keep only recent readings
        self.concentration_history.append(concentration)
        if len(self.concentration_history) > self.memory_length:
            self.concentration_history.pop(0)

        return concentration

    def calculate_gradient_response(self):
        """
        Calculate the bacterium's response to the sensed gradient.
        Returns a modified tumble rate based on the concentration change.
        """

        # Need at least two readings to detect a change
        if len(self.concentration_history) < 2:
            return self.tumble_rate

        # Calculate the change in concentration
        concentration_change = self.concentration_history[-1] - self.concentration_history[0]

        # Adjust tumble rate based on concentration change
        if concentration_change > 0: # Moving toward higher concentration
            # Decrease tumble rate to continue in this direction
            return max(0.01, self.tumble_rate - (concentration_change * self.sensitivity))
        else: # Moving toward lower concentration
            # Increase tumble rate to change direction
            return min(0.8, self.tumble_rate + (abs(concentration_change) * self.sensitivity))

    def tumble(self):
        """
        Execute a tumbling motion - randomly change direction.
        """

        self.state = "tumble"
        self.tumble_timer = 1 # Tumble for one step
        self.heading = random.uniform(0, 2 * math.pi) # Choose a random new direction

    def run(self):
        """
        Execute a run motion - move in the current direction.
        """

        self.state = "run"

        # Calculate new position based on heading and speed
        new_x = self.pos[0] + math.cos(self.heading) * self.speed
        new_y = self.pos[1] + math.sin(self.heading) * self.speed

        # Move the bacterium (ContinuousSpace handles boundary conditions)
        self.model.space.move_agent(self, (new_x, new_y))

    def step(self):
        """
        Defines what the agent does in one step of the simulation.
        """

        # Sense the environment
        self.sense_environment()

        # If currently tumbling, decrement timer
        if self.state == "tumble":
            self.tumble_timer -= 1
            if self.tumble_timer <= 0:
                self.state = "run"

        # Decide whether to run or tumble
        if self.state == "run":

            # Calculate tumble probability based on gradient
            tumble_probability = self.calculate_gradient_response()

            # Randomly decide whether to tumble
            if random.random() < tumble_probability:
                self.tumble()
            else:
                self.run()

        # Otherwise, continue tumbling

In [ ]:
# Define the chemotaxis model
class BacterialChemotaxisModel(Model):
    """
    A model simulating bacterial chemotaxis in a chemical gradient.
    """

    def __init__(self, width=50, height=50, n_bacteria=100, gradient_type="linear", steepness=1.0):
        """
        Initialize a new bacterial chemotaxis model.

        Args:
            width: Width of the space
            height: Height of the space
            n_bacteria: Initial number of bacteria
            gradient_type: Type of gradient ('linear', 'radial', or 'exponential')
            steepness: How steep the gradient is
        """

        super().__init__()
        self.space = ContinuousSpace(width, height, True) # Torus=True means the space wraps around
        self.chemical_gradient = ChemicalGradient(width, height, gradient_type, steepness=steepness)

        # Create and place bacteria
        for _ in range(n_bacteria):
            # Create bacterium with random parameters
            speed = random.uniform(0.3, 0.7)
            memory_length = random.randint(2, 5)
            sensitivity = random.uniform(3, 7)

            bacterium = ChemotacticBacterium(self, speed, memory_length, sensitivity)

            # Place the bacterium in a random position
            x = self.random.uniform(0, self.space.x_max)
            y = self.random.uniform(0, self.space.y_max)
            self.space.place_agent(bacterium, (x, y))

        # Add data collector to track metrics
        self.datacollector = DataCollector(
            model_reporters={
                "Mean Concentration": self.mean_concentration,
                "Concentration Variance": self.concentration_variance,
                "X Distribution": lambda m: self.position_distribution(m, 'x'),
                "Y Distribution": lambda m: self.position_distribution(m, 'y')
            }
        )

    def mean_concentration(self):
        """Calculate the mean concentration experienced by bacteria."""
        concentrations = [self.chemical_gradient.concentration_at(b.pos) for b in self.agents]
        return sum(concentrations) / len(concentrations) if concentrations else 0

    def concentration_variance(self):
        """Calculate the variance in concentration experienced by bacteria."""
        concentrations = [self.chemical_gradient.concentration_at(b.pos) for b in self.agents]
        if not concentrations:
            return 0
        mean = sum(concentrations) / len(concentrations)
        return sum((c - mean) ** 2 for c in concentrations) / len(concentrations)

    def position_distribution(self, model, axis):
        """Return a list of positions along the specified axis."""
        if axis == 'x':
            return [b.pos[0] for b in self.agents]
        else:
            return [b.pos[1] for b in self.agents]

    def step(self):
        """
        Advance the model by one step.
        """
        self.datacollector.collect(self)
        self.agents.shuffle_do("step")


## Visualization


In [ ]:
# Function to run the model and visualize it
def run_chemotaxis_model(width=50, height=50, n_bacteria=100, gradient_type="linear", steepness=1.0, steps=100):
    """
    Run the bacterial chemotaxis model and visualize it.

    Args:
        width: Width of the space
        height: Height of the space
        n_bacteria: Initial number of bacteria
        gradient_type: Type of gradient ('linear', 'radial', or 'exponential')
        steepness: How steep the gradient is
        steps: Number of steps to run the simulation
    """

    # Create a new model
    model = BacterialChemotaxisModel(width=width, height=height, n_bacteria=n_bacteria, gradient_type=gradient_type, steepness=steepness)

    # Create a grid to represent the chemical gradient
    gradient_grid = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            gradient_grid[y, x] = model.chemical_gradient.concentration_at((x, y))

    # Store all positions for animation
    all_positions = []
    for i in range(steps):
        model.step()
        positions = [(agent.pos[0], agent.pos[1]) for agent in model.agents]
        all_positions.append(positions)

    # Run the model for the specified number of steps
    for i in range(steps):
        model.step()

        # Save positions at each step
        positions = [(agent.pos[0], agent.pos[1]) for agent in model.agents]

    # Animate the simulation
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.imshow(gradient_grid, cmap='viridis', origin='lower', alpha=0.7, extent=[0, width, 0, height])
    scat = ax.scatter([], [], c='red', s=20, alpha=0.7)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_title("Bacterial Chemotaxis Trajectory")
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")

    def update(frame):
        positions = all_positions[frame]
        x_positions = [pos[0] for pos in positions]
        y_positions = [pos[1] for pos in positions]
        scat.set_offsets(np.column_stack((x_positions, y_positions)))
        ax.set_title(f"Bacterial Chemotaxis Trajectory (Step {frame+1})")
        return scat,

    ani = animation.FuncAnimation(fig, update, frames=len(all_positions), interval=100, blit=True)
    plt.close(fig)
    display(HTML(ani.to_jshtml()))

    # Get the collected data and create analysis plots
    model_data = model.datacollector.get_model_vars_dataframe()

    # Plot mean concentration over time
    plt.figure(figsize=(10, 6))
    plt.plot(model_data["Mean Concentration"])
    plt.title("Mean Chemical Concentration Experienced by Bacteria")
    plt.xlabel("Step")
    plt.ylabel("Mean Concentration")
    plt.grid(True)
    plt.show()

    # Plot position distributions at final step
    plt.figure(figsize=(15, 6))

    # X-position distribution
    plt.subplot(1, 2, 1)
    plt.hist(model_data["X Distribution"].iloc[-1], bins=20)
    plt.title("Final X-Position Distribution")
    plt.xlabel("X Position")
    plt.ylabel("Number of Bacteria")

    # Y-position distribution
    plt.subplot(1, 2, 2)
    plt.hist(model_data["Y Distribution"].iloc[-1], bins=20)
    plt.title("Final Y-Position Distribution")
    plt.xlabel("Y Position")
    plt.ylabel("Number of Bacteria")
    plt.tight_layout()
    plt.show()

    return model_data

## Interactive Experiments

In [ ]:
@interact(
    width=IntSlider(min=20, max=100, step=10, value=50, description='Space Width:'),
    height=IntSlider(min=20, max=100, step=10, value=50, description='Space Height:'),
    n_bacteria=IntSlider(min=10, max=500, step=10, value=100, description='Bacteria Count:'),
    gradient_type=Dropdown(
        options=['linear', 'radial', 'exponential'],
        value='radial',
        description='Gradient Type:'
    ),
    steepness=FloatSlider(min=0.2, max=5.0, step=0.2, value=1.0, description='Steepness:'),
    steps=IntSlider(min=20, max=200, step=20, value=100, description='Steps:')
)
def run_interactive_chemotaxis(width, height, n_bacteria, gradient_type, steepness, steps):
    return run_chemotaxis_model(width, height, n_bacteria, gradient_type, steepness, steps)